In [1]:
#import stackview
import numpy as np
import skimage.filters as filters
import skimage.measure as measure
import skimage.feature as feature
import skimage.segmentation as segmentation
import scipy.ndimage as ndi
import matplotlib.pyplot as plt

In [2]:
img_dims = 256

In [3]:
img = np.zeros([img_dims, img_dims, img_dims])

In [4]:
pts = np.random.randint(0, img_dims, [1000, 3])

In [5]:
img[pts[:,0], pts[:,1], pts[:,2]] = 128

In [6]:
blurred = filters.gaussian(img, sigma=5)

In [7]:
noisy = blurred + np.random.normal(0, np.max(blurred)/100.0, img.shape)

In [8]:
import napari
viewer = napari.Viewer()

stackview.slice(noisy, continuous_update=True)

In [9]:
noisy.shape

(256, 256, 256)

In [10]:
viewer.add_image(noisy)

<Image layer 'noisy' at 0x7f8d94bd20d0>

# Threshold and segment

In [11]:
thresh = np.percentile(noisy, 90)

In [12]:
noisy_blurred = filters.gaussian(noisy, sigma=2)

In [13]:
mask = noisy_blurred > thresh

In [17]:
edt = ndi.distance_transform_edt(mask)

In [15]:
viewer.add_image(edt)

<Image layer 'edt' at 0x7f8d7d033400>

In [18]:
smoothed_edt = filters.gaussian(edt, sigma=0.1)

In [19]:
viewer.add_image(smoothed_edt)

<Image layer 'smoothed_edt' at 0x7f8d05f66580>

In [20]:
maxes = feature.peak_local_max(smoothed_edt, min_distance=3)

In [21]:
viewer.add_points(maxes, size=5, n_dimensional=True)

<Points layer 'maxes' at 0x7f8d7d25d2b0>

In [22]:
peak_image = np.zeros(img.shape)
peak_image[maxes[:,0], maxes[:,1], maxes[:,2]] = 1
peak_image = measure.label(peak_image)

In [23]:
viewer.add_labels(peak_image)

<Labels layer 'peak_image' at 0x7f8d05fac130>

In [24]:
watershedded = segmentation.watershed(-smoothed_edt,peak_image, mask=mask)

In [25]:
viewer.add_labels(watershedded)

<Labels layer 'watershedded' at 0x7f8d7d25ddf0>

# Testing expand_labels

In [52]:
%%time
rslt = segmentation.expand_labels(watershedded, 8)

CPU times: user 3.57 s, sys: 591 ms, total: 4.16 s
Wall time: 5.09 s


In [45]:
import scipy.ndimage as ndi
def expand_labels(labels, distance=1, spacing=None):
    """Expand labels by a given distance in each dimension.
    
    Args:
        labels (np.ndarray): Labels to expand.
        distance (int): Distance to expand labels.
        spacing (list): Spacing of labels in each dimension.
        
    Returns:
        np.ndarray: Expanded labels.
    """
    if spacing is None:
        spacing = [1] * len(labels.shape)
    
    edt, idx = ndi.distance_transform_edt(labels==0, return_indices=True, sampling=spacing)
    expanded = labels[tuple(idx)] * (edt <= distance)
    return expanded

In [54]:
%%time
expanded = expand_labels(watershedded, distance=3, spacing=[1, 1, 1])

CPU times: user 3.65 s, sys: 484 ms, total: 4.13 s
Wall time: 4.16 s


In [55]:
viewer.add_labels(expanded)

<Labels layer 'expanded [2]' at 0x7f8d06803160>